In [1]:

## annual_electricity_generation_emission_v1.0

## raw data: 
## 1. Australia_emission.xlsx
## 2. australian_energy_update_report_data_for_charts_2018.xlsx

## result: australia_electricity_generation.csv
## columns: 
#### year -- calender 
#### electricity_generation_energy -- unit is PJ, 1PJ = 278 million kWh.
#### co2_emission_amount -- unit is Mt, million tones
#### co2_emission_per_generation -- unit is kg/w
#### update_time

## main idea:
## 1. select the electricity generation data 
## 2. select the electricity co2 emission
## 3. join them on calendar year
## 4. calculate "co2_emission_per_generation"


In [ ]:

## import the needed library
import pandas as pd 
import time
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.api import tsa


In [3]:
################### part1 -- select the electricity generation data #######################

## read the excel file
electricity_generation_excel_data = pd.ExcelFile('/Users/cuimingyue/Documents/5120/code/wrangling_data/data_wrangling_v5.0_06:05:2019/raw data/australian_energy_update_report_data_for_charts_2018.xlsx')
## you can use code "excel_data.sheet_names" to check the sheetname


## choose one sheet
electricity_generation = electricity_generation_excel_data.parse('2.4')

## delect some "missing value" rows and columns
electricity_generation = electricity_generation.dropna(0, how = 'any')
electricity_generation = electricity_generation.dropna(1, how = 'all')

## rerange the index
electricity_generation.index = range(len(electricity_generation.index))

## check the "australia_generation"
electricity_generation


Figure 2.4: Australian energy consumption, by sector  Unnamed: 1  \
0                                             1975–76         559.9   
1                                             1976–77         634.3   
2                                             1977–78         662.6   
3                                             1978–79         691.6   
4                                             1979–80         743.3   
5                                             1980–81         778.2   
6                                             1981–82         845.3   
7                                             1982–83         829.7   
8                                             1983–84         846.6   
9                                             1984–85         900.9   
10                                            1985–86         896.2   
11                                            1986–87         939.5   
12                                            1987–88         962.5   
13                                            1988–89        1047.6   
14                                            1989–90          1066   
15                                            1990–91          1065   
16                                            1991–92        1092.9   
17                                            1992–93        1096.4   
18                                            1993–94        1109.6   
19                                            1994–95        1162.6   
20                                            1995–96        1211.8   
21                                            1996–97        1244.1   
22                                            1997–98        1347.3   
23                                            1998–99        1415.6   
24                                            1999–00        1427.4   
25                                            2000–01        1507.9   
26                                            2001–02        1542.9   
27                                            2002–03        1566.7   
28                                            2003–04        1664.8   
29                                            2004–05        1706.4   
30                                            2005–06        1730.7   
31                                            2006–07        1807.8   
32                                            2007–08        1823.9   
33                                            2008–09        1916.1   
34                                            2009–10        1759.2   
35                                            2010–11        1687.1   
36                                            2011–12        1727.5   
37                                            2012–13        1614.7   
38                                            2013–14        1576.7   
39                                            2014–15        1670.1   
40                                            2015–16        1727.7   
41                                            2016–17        1692.7   

   Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7  
0         715      925.9       67.3       89.6      246.4      126.7  
1       761.9      943.1       73.8       93.9      260.2      138.7  
2       798.7      952.9         73       96.1        261      140.8  
3       812.9      955.1       76.5      100.3      270.5      146.2  
4       824.6      965.2       80.8      104.4      261.9      151.2  
5       835.2        946       71.2      106.6      262.8      146.3  
6       843.8      939.5       71.3      109.5      272.8      155.4  
7         837      852.7       75.1      110.2      273.2      144.8  
8       864.1      887.8       83.2      110.7        276      152.8  
9       892.1      934.3       96.9        115      282.1      149.4  
10      906.3      939.6       97.9      121.6      288.4      152.5  
11      922.1        966        105      127.8      298.1      155.9  
12        966      988.3  

In [4]:

## create the new dataframe to store the "calender", "electricity_generation_energy"
new_electricity_generation = pd.DataFrame(columns = ['year','electricity_generation']) 

## input the related value into the new dataframe
new_electricity_generation['year'] = electricity_generation.iloc[:,0]
new_electricity_generation['electricity_generation'] = electricity_generation.iloc[:,1]

## check the "australia_generation"
new_electricity_generation


year electricity_generation
0   1975–76                  559.9
1   1976–77                  634.3
2   1977–78                  662.6
3   1978–79                  691.6
4   1979–80                  743.3
5   1980–81                  778.2
6   1981–82                  845.3
7   1982–83                  829.7
8   1983–84                  846.6
9   1984–85                  900.9
10  1985–86                  896.2
11  1986–87                  939.5
12  1987–88                  962.5
13  1988–89                 1047.6
14  1989–90                   1066
15  1990–91                   1065
16  1991–92                 1092.9
17  1992–93                 1096.4
18  1993–94                 1109.6
19  1994–95                 1162.6
20  1995–96                 1211.8
21  1996–97                 1244.1
22  1997–98                 1347.3
23  1998–99                 1415.6
24  1999–00                 1427.4
25  2000–01                 1507.9
26  2001–02                 1542.9
27  2002–03                 1566.7
28  2003–04                 1664.8
29  2004–05                 1706.4
30  2005–06                 1730.7
31  2006–07                 1807.8
32  2007–08                 1823.9
33  2008–09                 1916.1
34  2009–10                 1759.2
35  2010–11                 1687.1
36  2011–12                 1727.5
37  2012–13                 1614.7
38  2013–14                 1576.7
39  2014–15                 1670.1
40  2015–16                 1727.7
41  2016–17                 1692.7

In [5]:

## as the raw data is financial year, it should change them into the calendar year

## initial the first year
## initial the "year" list
initial_year = 1975
year = []
year.append(initial_year)


##calculate the row number for "for loop"
row_num = len(new_electricity_generation)


## insert the year value to the year colunmn
for i in range(row_num):
    initial_year = initial_year + 1
    year.append(initial_year)
    

## change the year list to the dataframe
## add the year dataframe to the electricity_generation dataframe
year = pd.DataFrame(year)
new_electricity_generation['calendar_year'] = year

## check the "australia_generation"
new_electricity_generation


year electricity_generation  calendar_year
0   1975–76                  559.9           1975
1   1976–77                  634.3           1976
2   1977–78                  662.6           1977
3   1978–79                  691.6           1978
4   1979–80                  743.3           1979
5   1980–81                  778.2           1980
6   1981–82                  845.3           1981
7   1982–83                  829.7           1982
8   1983–84                  846.6           1983
9   1984–85                  900.9           1984
10  1985–86                  896.2           1985
11  1986–87                  939.5           1986
12  1987–88                  962.5           1987
13  1988–89                 1047.6           1988
14  1989–90                   1066           1989
15  1990–91                   1065           1990
16  1991–92                 1092.9           1991
17  1992–93                 1096.4           1992
18  1993–94                 1109.6           1993
19  1994–95                 1162.6           1994
20  1995–96                 1211.8           1995
21  1996–97                 1244.1           1996
22  1997–98                 1347.3           1997
23  1998–99                 1415.6           1998
24  1999–00                 1427.4           1999
25  2000–01                 1507.9           2000
26  2001–02                 1542.9           2001
27  2002–03                 1566.7           2002
28  2003–04                 1664.8           2003
29  2004–05                 1706.4           2004
30  2005–06                 1730.7           2005
31  2006–07                 1807.8           2006
32  2007–08                 1823.9           2007
33  2008–09                 1916.1           2008
34  2009–10                 1759.2           2009
35  2010–11                 1687.1           2010
36  2011–12                 1727.5           2011
37  2012–13                 1614.7           2012
38  2013–14                 1576.7           2013
39  2014–15                 1670.1           2014
40  2015–16                 1727.7           2015
41  2016–17                 1692.7           2016

In [6]:

## calculate the related electricity generation of calendar year

## initial the "calender year electricity generation" list
## add the first value to the list
calendar_year_elec_gener = []
calendar_year_elec_gener.append(new_electricity_generation.iloc[0,1]/2)
calendar_year_elec_gener


## insert "calender year electricity generation" value to the relevant colunmn
for i in range(row_num-1):
    j = i+1
    value = (new_electricity_generation.iloc[j,1] + new_electricity_generation.iloc[j-1,1])/2
    calendar_year_elec_gener.append(value)
    

## change the "calender year electricity generation" list to the dataframe
## add the year dataframe to the electricity_generation dataframe
calendar_year_elec_gener = pd.DataFrame(calendar_year_elec_gener)
new_electricity_generation['calendar_year_elec_gener'] = calendar_year_elec_gener


## check the "australia_generation"
new_electricity_generation



year electricity_generation  calendar_year  calendar_year_elec_gener
0   1975–76                  559.9           1975                    279.95
1   1976–77                  634.3           1976                    597.10
2   1977–78                  662.6           1977                    648.45
3   1978–79                  691.6           1978                    677.10
4   1979–80                  743.3           1979                    717.45
5   1980–81                  778.2           1980                    760.75
6   1981–82                  845.3           1981                    811.75
7   1982–83                  829.7           1982                    837.50
8   1983–84                  846.6           1983                    838.15
9   1984–85                  900.9           1984                    873.75
10  1985–86                  896.2           1985                    898.55
11  1986–87                  939.5           1986                    917.85
12  1987–88                  962.5           1987                    951.00
13  1988–89                 1047.6           1988                   1005.05
14  1989–90                   1066           1989                   1056.80
15  1990–91                   1065           1990                   1065.50
16  1991–92                 1092.9           1991                   1078.95
17  1992–93                 1096.4           1992                   1094.65
18  1993–94                 1109.6           1993                   1103.00
19  1994–95                 1162.6           1994                   1136.10
20  1995–96                 1211.8           1995                   1187.20
21  1996–97                 1244.1           1996                   1227.95
22  1997–98                 1347.3           1997                   1295.70
23  1998–99                 1415.6           1998                   1381.45
24  1999–00                 1427.4           1999                   1421.50
25  2000–01                 1507.9           2000                   1467.65
26  2001–02                 1542.9           2001                   1525.40
27  2002–03                 1566.7           2002                   1554.80
28  2003–04                 1664.8           2003                   1615.75
29  2004–05                 1706.4           2004                   1685.60
30  2005–06                 1730.7           2005                   1718.55
31  2006–07                 1807.8           2006                   1769.25
32  2007–08                 1823.9           2007                   1815.85
33  2008–09                 1916.1           2008                   1870.00
34  2009–10                 1759.2           2009                   1837.65
35  2010–11                 1687.1           2010                   1723.15
36  2011–12                 1727.5           2011                   1707.30
37  2012–13                 1614.7           2012                   1671.10
38  2013–14                 1576.7           2013                   1595.70
39  2014–15                 1670.1           2014                   1623.40
40  2015–16                 1727.7           2015                   1698.90
41  2016–17                 1692.7           2016                   1710.20

In [7]:
## drop the 1st row and keep calender year and related generation value
new_electricity_generation = new_electricity_generation.iloc[1:,2:4]
new_electricity_generation.index = range(len(new_electricity_generation.index))


## rename the column names
new_electricity_generation.rename(columns={'calendar_year':'year', 'calendar_year_elec_gener':'electricity_generation'}, inplace = True)


## change the data type of dataframe value
new_electricity_generation['electricity_generation'] = new_electricity_generation['electricity_generation'].astype('str')
new_electricity_generation['year'] = new_electricity_generation['year'].astype('str')


## check the "australia_generation"
new_electricity_generation


year electricity_generation
0   1976      597.0999999999999
1   1977                 648.45
2   1978                  677.1
3   1979                 717.45
4   1980                 760.75
5   1981                 811.75
6   1982                  837.5
7   1983      838.1500000000001
8   1984                 873.75
9   1985                 898.55
10  1986                 917.85
11  1987                  951.0
12  1988                1005.05
13  1989                 1056.8
14  1990                 1065.5
15  1991                1078.95
16  1992                1094.65
17  1993                 1103.0
18  1994                 1136.1
19  1995     1187.1999999999998
20  1996     1227.9499999999998
21  1997     1295.6999999999998
22  1998     1381.4499999999998
23  1999                 1421.5
24  2000                1467.65
25  2001                 1525.4
26  2002     1554.8000000000002
27  2003                1615.75
28  2004                 1685.6
29  2005     1718.5500000000002
30  2006                1769.25
31  2007                1815.85
32  2008                 1870.0
33  2009                1837.65
34  2010                1723.15
35  2011                 1707.3
36  2012                 1671.1
37  2013                 1595.7
38  2014                 1623.4
39  2015                 1698.9
40  2016                 1710.2

In [8]:

################### part2 -- select the electricity generation data #######################

## read the excel file
electricity_emission_excel_data = pd.ExcelFile('/Users/cuimingyue/Documents/5120/code/wrangling_data/data_wrangling_v5.0_06:05:2019/raw data/Australia_emission.xlsx')
## you can use code "electricity_emission_excel_data.sheet_names" to check the sheetname

## choose one sheet
australia_emission = electricity_emission_excel_data.parse('Figure 5')

## delect some "missing value" rows and columns
australia_emission = australia_emission.dropna(0, how = 'any')
australia_emission = australia_emission.dropna(1, how = 'any')
australia_emission.index = range(len(australia_emission.index))

## delect not meanningfule rows
australia_emission = australia_emission.drop(australia_emission.index[0:3])
australia_emission.index = range(len(australia_emission.index))

## check the "australia_emission"
australia_emission



Quarterly Update of Australia's National Greenhouse Gas Inventory | Figures and Tables for the September Quarter 2018  \
0                                                1991                                                                      
1                                                1992                                                                      
2                                                1993                                                                      
3                                                1994                                                                      
4                                                1995                                                                      
5                                                1996                                                                      
6                                                1997                                                                      
7                                                1998                                                                      
8                                                1999                                                                      
9                                                2000                                                                      
10                                               2001                                                                      
11                                               2002                                                                      
12                                               2003                                                                      
13                                               2004                                                                      
14                                               2005                                                                      
15                                               2006                                                                      
16                                               2007                                                                      
17                                               2008                                                                      
18                                               2009                                                                      
19                                               2010                                                                      
20                                               2011                                                                      
21                                               2012                                                                      
22                                               2013                                                                      
23                                               2014                                                                      
24                                               2015                                                                      
25                                               2016                                                                      
26                                               2017                                                                      
27                                               2018                                                                      

   Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6  \
0         2.3        0.6       -0.1        0.7       -0.4       -1.1   
1         4.7        1.4        1.1        2.1       -0.1       -3.2   
2         5.8        2.8        2.6       -0.1       -0.3       -4.9   
3         8.4        3.9        4.6         -1       -0.2       -5.4   
4        13.9        4.5        7.6       -0.3

In [9]:

## create the new dataframe to store the "calender", "electricity_generation_energy"
electricity_emission = pd.DataFrame(columns = ['year','electricity_co2_emission'])

## input the related value into the new dataframe
electricity_emission['year'] = australia_emission.iloc[:,0].astype('str')
electricity_emission['electricity_co2_emission'] = australia_emission.iloc[:,1].astype('str')

## check the "electricity_emission"
electricity_emission



year electricity_co2_emission
0   1991                      2.3
1   1992                      4.7
2   1993                      5.8
3   1994                      8.4
4   1995                     13.9
5   1996                     18.9
6   1997                     25.9
7   1998                     36.9
8   1999                     42.2
9   2000                     45.6
10  2001                     54.8
11  2002                     54.9
12  2003                     56.8
13  2004                     66.7
14  2005                     66.6
15  2006                     72.5
16  2007                     75.3
17  2008                     77.7
18  2009                     77.5
19  2010                     74.6
20  2011                     68.3
21  2012                     65.1
22  2013                     54.9
23  2014                     53.1
24  2015                     60.7
25  2016                     63.9
26  2017                     56.2
27  2018                     50.3

In [10]:

######################## part3 -- join them on calendar year ################################## 
######################## part4 -- calculate "co2_emission_per_generation" ################ 
## one dataframe is: new_electricity_generation
## another dataframe is: electricity_emission

## mege 2 dataframe -- new_electricity_generation + electricity_emission
annual_electricity_generation_emission = pd.merge(new_electricity_generation, electricity_emission, how='outer')

## delect some "missing value" rows and columns
annual_electricity_generation_emission = annual_electricity_generation_emission.dropna(0, how = 'any')
annual_electricity_generation_emission.index = range(len(annual_electricity_generation_emission.index))

## calculate "co2_emission_per_generation" 
annual_electricity_generation_emission['co2_emission_per_generation'] = (annual_electricity_generation_emission['electricity_co2_emission'].astype('float')*1000)/(annual_electricity_generation_emission['electricity_generation'].astype('float')*278)

## add the update_time column
annual_electricity_generation_emission['update_time'] = time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()))


## check the "annual_electricity_generation_emission"
annual_electricity_generation_emission


## set the index column name
annual_electricity_generation_emission.index.name = 'annual_electricity_id' 

## check the "annual_electricity_emission"
annual_electricity_generation_emission

year electricity_generation electricity_co2_emission  \
annual_electricity_id                                                         
0                      1991                1078.95                      2.3   
1                      1992                1094.65                      4.7   
2                      1993                 1103.0                      5.8   
3                      1994                 1136.1                      8.4   
4                      1995     1187.1999999999998                     13.9   
5                      1996     1227.9499999999998                     18.9   
6                      1997     1295.6999999999998                     25.9   
7                      1998     1381.4499999999998                     36.9   
8                      1999                 1421.5                     42.2   
9                      2000                1467.65                     45.6   
10                     2001                 1525.4                     54.8   
11                     2002     1554.8000000000002                     54.9   
12                     2003                1615.75                     56.8   
13                     2004                 1685.6                     66.7   
14                     2005     1718.5500000000002                     66.6   
15                     2006                1769.25                     72.5   
16                     2007                1815.85                     75.3   
17                     2008                 1870.0                     77.7   
18                     2009                1837.65                     77.5   
19                     2010                1723.15                     74.6   
20                     2011                 1707.3                     68.3   
21                     2012                 1671.1                     65.1   
22                     2013                 1595.7                     54.9   
23                     2014                 1623.4                     53.1   
24                     2015                 1698.9                     60.7   
25                     2016                 1710.2                     63.9   

                       co2_emission_per_generation          update_time  
annual_electricity_id                                                    
0                                         0.007668  2019-05-08 23:37:39  
1                                         0.015445  2019-05-08 23:37:39  
2                                         0.018915  2019-05-08 23:37:39  
3                                         0.026596  2019-05-08 23:37:39  
4                                         0.042116  2019-05-08 23:37:39  
5                                         0.055365  2019-05-08 23:37:39  
6                                         0.071904  2019-05-08 23:37:39  
7                                         0.096083  2019-05-08 23:37:39  
8                                         0.106788  2019-05-08 23:37:39  
9                                         0.111763  2019-05-08 23:37:39  
10                                        0.129227  2019-05-08 23:37:39  
11                                        0.127014  2019-05-08 23:37:39  
12                                        0.126453  2019-05-08 23:37:39  
13                                        0.142340  2019-05-08 23:37:39  
14                                        0.139401  2019-05-08 23:37:39  
15                                        0.147402  2019-05-08 23:37:39  
16                                        0.149166  2019-05-08 23:37:39  
17                                        0.149463  2019-05-08 23:37:39  
18                                        0.151703  2019-05-08 23:37:39  
19                                        0.155730  2019-05-08 23:37:39  
20                                        0.143902  2019-05-08 23:37:39  
21                                        0.140131  2019-05-08 23:37:39  
22                                

In [11]:

## write out the “australia_electricity_generation” csv files
annual_electricity_generation_emission.to_csv('/Users/cuimingyue/Documents/5120/code/wrangling_data/data_wrangling_v5.0_06:05:2019/result table/annual_electricity_generation_emission.csv', encoding='utf-8')
